In [3]:
%reload_ext autoreload
%autoreload 2
import functions
from functions import *
import os
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
#import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', 20)
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import joblib
import os

# Data
## Preprocessing data

In [4]:
data_transformed_full = phase1(debug = True, num_rows=25000000)

Train samples: 307511, test samples: 48744
Bureau df shape: (305808, 116)
Process bureau and bureau_balance - done in 31s
Previous applications df shape: (338857, 249)
Process previous_applications - done in 52s
Pos-cash balance df shape: (337252, 18)
Process POS-CASH balance - done in 29s
Installments payments df shape: (339587, 26)
Process installments payments - done in 53s
Credit card balance df shape: (103558, 141)
Process credit card balance - done in 38s
data avant remplissage shape : (356251, 809)
data final shape : (356251, 809)


In [8]:
data_transformed_full.to_csv('data_transformed_full.csv')

In [4]:
data_transformed = phase1(debug = True, num_rows=10000)

Train samples: 10000, test samples: 10000
Bureau df shape: (2011, 108)
Process bureau and bureau_balance - done in 1s
Previous applications df shape: (9734, 242)
Process previous_applications - done in 1s
Pos-cash balance df shape: (9494, 15)
Process POS-CASH balance - done in 0s
Installments payments df shape: (8893, 26)
Process installments payments - done in 0s
Credit card balance df shape: (9520, 131)
Process credit card balance - done in 0s
data avant remplissage shape : (20000, 779)
data final shape : (20000, 257)


In [5]:
data_transformed.to_csv('data_transformed.csv')

# Split in train and test pour le modèle final

In [6]:
data_transformed_full = pd.read_csv('data_transformed_full.csv').drop('Unnamed: 0', axis=1)

In [7]:
def train_test(df):
    feats = [f for f in df.columns if f not in ['SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
    df1 = df[feats]
    # Divide in training and test data
    X_train = df1[df1['TARGET'].notnull()].drop('TARGET', axis=1)
    y_train = df1[df1['TARGET'].notnull()]['TARGET']
    X_test = df1[df1['TARGET'].isna()].drop('TARGET', axis=1)
    
    return (X_train, X_test, y_train)

In [8]:
X_train, X_test, y_train= train_test(data_transformed_full)

In [6]:
X_train.shape

(307507, 806)

In [12]:
# Creation d'un fichier X_train non complété pour le dashboard
X_train.to_csv('X_train_with_nan.csv')

In [14]:
# Creation d'un fichier X_test non complété pour le dashboard
some_customers_with_nan = X_test.copy()
some_customers_with_nan['SK_ID_CURR'] = data_transformed_full[data_transformed_full['TARGET'].isna()]['SK_ID_CURR'].values
some_customers_with_nan.iloc[:5000].to_csv('some_customers_with_nan.csv')


In [ ]:
def missing_values(X_train, X_test, df):
    feats = [f for f in df.columns if f not in ['SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index', 'TARGET']]
    imp = SimpleImputer(strategy='median')
    imp.fit(X_train)
    X_train_full = imp.transform(X_train)
    X_test_full = imp.transform(X_test)
    X_train_full = pd.DataFrame(columns=feats, data=X_train_full)
    X_test_full = pd.DataFrame(columns=feats, data=X_test_full)
    return X_train_full, X_test_full

In [ ]:
X_train_full, X_test_full = missing_values(X_train, X_test, data_transformed_full)

In [33]:
X_train_full.to_csv('X_train.csv')     
X_test_full.to_csv('X_test.csv')     
y_train.to_csv('y_train.csv')

In [40]:
data_transformed_full[data_transformed_full['TARGET'].isna()]['SK_ID_CURR']

307507    100001
307508    100005
307509    100013
307510    100028
307511    100038
           ...  
356246    456221
356247    456222
356248    456223
356249    456224
356250    456250
Name: SK_ID_CURR, Length: 48744, dtype: int64

In [42]:
# Creation et sauvegarde d'un sous-tableau test 
customers = X_test_full.copy()
customers['SK_ID_CURR'] = data_transformed_full[data_transformed_full['TARGET'].isna()]['SK_ID_CURR'].values

In [43]:
customers.to_csv('customers_test.csv')              
some_customers = customers.iloc[:5000]
some_customers.to_csv('some_customers.csv')  

In [44]:
# Creation et sauvegarde d'une liste de features
features = pd.DataFrame(X_train_full.columns).rename(columns={0: 'features'})
features.to_csv('features.csv')

In [2]:
some_customers = pd.read_csv('some_customers.csv')

In [3]:
some_customers

,Unnamed: 0,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT,SK_ID_CURR
0,0,0.0,1.0,0.0,0.0,0.0,135000.0,568800.0,20560.5,450000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,100001
1,1,0.0,0.0,0.0,0.0,0.0,99000.0,222768.0,17370.0,180000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,100005
2,2,0.0,0.0,1.0,0.0,0.0,202500.0,663264.0,69777.0,630000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,96.0,100013
3,3,0.0,1.0,0.0,0.0,2.0,315000.0,1575000.0,49018.5,1575000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49.0,100028
4,4,0.0,0.0,1.0,1.0,1.0,180000.0,625500.0,32067.0,625500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,100038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4995,0.0,1.0,0.0,0.0,0.0,157500.0,375367.5,38592.0,337500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,136103
4996,4996,0.0,1.0,0.0,0.0,1.0,67500.0,450000.0,15498.0,450000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,136114
4997,4997,0.0,1.0,0.0,0.0,1.0,112500.0,707287.5,34155.0,562500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,136116
4998,4998,0.0,1.0,0.0,1.0,0.0,202500.0,848745.0,36090.0,675000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,96.0,136140
